In [1]:
with open('../input.txt','r') as f:
    text = f.read()
data = text[:1000]
print(data[:100])  # Print the first 100 characters

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [2]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode(data)
print(tokens[:24])

[5962, 22307, 25, 198, 8421, 356, 5120, 597, 2252, 11, 3285, 502, 2740, 13, 198, 198, 3237, 25, 198, 5248, 461, 11, 2740, 13]


In [45]:
import torch
token = torch.randint(0, 10, (1,100), dtype=torch.long)[0]
token

tensor([2, 5, 5, 2, 7, 1, 0, 1, 6, 7, 4, 3, 1, 4, 8, 9, 0, 3, 3, 7, 1, 7, 5, 6,
        3, 2, 2, 9, 5, 5, 8, 5, 5, 5, 7, 2, 5, 6, 8, 3, 1, 5, 4, 8, 8, 1, 5, 7,
        1, 6, 1, 3, 1, 4, 4, 0, 4, 9, 2, 5, 4, 2, 4, 9, 5, 1, 9, 9, 0, 6, 5, 2,
        4, 3, 4, 0, 6, 0, 1, 4, 7, 3, 6, 8, 8, 6, 0, 4, 6, 5, 2, 7, 5, 2, 5, 4,
        9, 8, 4, 1])

In [ ]:
buf = torch.tensor(tokens[:24 + 1])
print(buf)
# print(buf[:-1])
# print(buf[1:])
x = buf[:-1].view(4,6)
y= buf[1:].view(4,6)
# y = buf
print(y)
print(x)

tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,
         3285,   502,  2740,    13,   198,   198,  3237,    25,   198,  5248,
          461,    11,  2740,    13,   198])
tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,
         3285,   502,  2740,    13,   198,   198,  3237,    25,   198,  5248,
          461,    11,  2740,    13])
tensor([22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,  3285,
          502,  2740,    13,   198,   198,  3237,    25,   198,  5248,   461,
           11,  2740,    13,   198])
tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])


In [2]:
import torch

# super simple little MLP
net = torch.nn.Sequential(
    torch.nn.Linear(16,32),
    torch.nn.GELU(),
    torch.nn.Linear(32,1)
)

torch.random.manual_seed(42)
x = torch.randn(4, 16)
y = torch.randn(4, 1)
net.zero_grad()
yhat = net(x)
loss = torch.nn.functional.mse_loss(yhat, y)
loss.backward()
print(net[0].weight.grad.view(-1)[:10])

# the loss objective here is (due to readuction='mean')
# L = 1/4 * [
#     (y[0] - yhat[0])**2 +
#     (y[1] - yhat[1])**2 +
#     (y[2] - yhat[2])**2 +
#     (y[3] - yhat[3])**2 
# ]
# NOTE: 1/4!

tensor([-0.0150,  0.0011,  0.0042, -0.0040,  0.0059, -0.0080, -0.0078, -0.0138,
        -0.0103, -0.0134])


In [4]:
# now let's do it with grad_accum_steps of 4, and B=1
# the loss objective here is different because
# accumulation in gradient <---> SUM in lossi.e. we instead get:
# L0 = (y[0] - yhat[0])**2
# L1 = (y[1] - yhat[1])**2
# L2 = (y[2] - yhat[2])**2
# L3 = (y[3] - yhat[3])**2
# L = L0 + L1 + L2 + L3
# NOTE: the "normalizer" of 1/4 is lost

net.zero_grad()
for i in range(4):
    yhat = net(x[i])
    loss = torch.nn.functional.mse_loss(yhat, y[i])
    loss.backward()
print("not normalized:",net[0].weight.grad.view(-1)[:10])

# L = 1/4(L0 + L1 + L2 + L3)
# NOTE: here accumulation steps is 4

net.zero_grad()
for i in range(4):
    yhat = net(x[i])
    loss = torch.nn.functional.mse_loss(yhat, y[i])
    loss = loss / 4
    loss.backward()
print("normalized:",net[0].weight.grad.view(-1)[:10])

not normalized: tensor([-0.0598,  0.0042,  0.0167, -0.0161,  0.0235, -0.0320, -0.0311, -0.0550,
        -0.0410, -0.0536])
normalized: tensor([-0.0150,  0.0011,  0.0042, -0.0040,  0.0059, -0.0080, -0.0078, -0.0138,
        -0.0103, -0.0134])
